In [1]:
import pandas as pd
import numpy as np
from benchmarks import MinVarianceMethod, MaxSharpeMethod
from environment import Environment
from utils import get_stats, cumulative_pct_change
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Loading Environments

In [ ]:
env = Environment()
minvar_method = MinVarianceMethod(allow_short=True)
maxsharpe_method = MaxSharpeMethod(allow_short=True)

In [ ]:
len(env.returns)

# Testing and getting results

In [ ]:
WINDOW_FIT = 180
WINDOW_HOLD = 180
N_ASSETS = len(env.returns.columns)
actions_equal, actions_minvar, actions_maxsharpe = [], [], []
result_equal, result_minvar, result_maxsharpe = [], [], []


for i in range(WINDOW_FIT, len(env.returns), WINDOW_HOLD):
    print(f"Testing Window from {i} to {i+WINDOW_FIT} and Training window from {i+WINDOW_FIT} to {i+WINDOW_FIT+WINDOW_HOLD}")

    if (i + WINDOW_FIT  > len(env.returns)) :
        print("Exceeded training and hold window, will stop")
        break

    state = env.get_state(
        start=i, 
        length=WINDOW_FIT
        )
    
    equal_weights = np.ones(N_ASSETS) / N_ASSETS
    min_var_weights = minvar_method.get_optimal_weights(state)
    print(f"Min var weights: {min_var_weights}")
    max_sharpe_weights = maxsharpe_method.get_optimal_weights(state)
    print(f"Max sharpe weights: {max_sharpe_weights}")

    holding_period_returns = env.get_state(
        start = i + WINDOW_FIT, 
        length = WINDOW_HOLD
    )
    
    r = np.dot(holding_period_returns, equal_weights)
    result_equal.append(r.tolist())
    actions_equal.append(equal_weights)
    
    r = np.dot(holding_period_returns, min_var_weights)
    result_minvar.append(r.tolist())
    actions_minvar.append(min_var_weights)

    r = np.dot(holding_period_returns, max_sharpe_weights)
    result_maxsharpe.append(r.tolist())
    actions_maxsharpe.append(max_sharpe_weights)


# Evaluation

In [ ]:
test = [0, -50, 50, ]
assert cumulative_pct_change(test).tolist() == [0,-50.0,-25.0]

In [ ]:
# flatten into list and compare results
equal_weights_daily_returns = [item for sublist in result_equal for item in sublist]
minvar_method_daily_returns = [item for sublist in result_minvar for item in sublist]
maxsharpe_method_daily_returns = [item for sublist in result_maxsharpe for item in sublist]

plt.figure()
plt.xlabel("Days", fontsize=12)
plt.ylabel("Returns (%)")
plt.plot(cumulative_pct_change(equal_weights_daily_returns), label = "Equal Weights Portfolio")
plt.plot(cumulative_pct_change(minvar_method_daily_returns), label = "Minimum Variance Portfolio")
plt.plot(cumulative_pct_change(maxsharpe_method_daily_returns), label = "Maximum Sharpe Portfolio")
plt.ylabel("Cumulative Returns (%)", fontsize=12)
plt.title("Portfolio Performance Comparison", fontsize=14)
plt.legend(fontsize=10)  # Add legend for clarity
plt.show()


In [ ]:
results = []


equal_weights_stats = get_stats(equal_weights_daily_returns)
equal_weights_stats["Name"] = "Equal Weights Portfolio"
results.append(equal_weights_stats)

minvar_method_stats = get_stats(minvar_method_daily_returns)
minvar_method_stats["Name"] = "Minimum Variance Portfolio"
results.append(minvar_method_stats)

maxsharpe_method_stats = get_stats(maxsharpe_method_daily_returns)
maxsharpe_method_stats["Name"] = "Maximum Sharpe Portfolio"
results.append(maxsharpe_method_stats)


stats = pd.DataFrame(results).set_index("Name")
stats